# One-card poker game with private information

In this tutorial, we'll create an extensive form representation of a one-card poker game ([Mye91](#mye91)) and use it to demonstrate and explain the following with Gambit:

1. Setting up an extensive form game with imperfect information
2. Using information sets
3. [Computing Nash equilibria](#)
4. [Acceptance criteria for Nash equilibria](#)

A version of this game also appears in [RUW08](#ruw08), as a classroom game under the name "stripped-down poker".

This is perhaps the simplest interesting game with imperfect information.

In our version of the game, there are two players, **Alice** and **Bob**, and a deck of cards, with equal numbers of **King** and **Queen** cards.

- The game begins with each player putting $1 in the pot.
- A card is dealt at random to Alice
    - Alice observes her card
    - Bob does not observe the card
- Alice then chooses either to **Raise** or to **Fold**.
    - If she chooses to Fold, Bob wins the pot and the game ends.
    - If she chooses to Raise, she adds another $1 to the pot.
- Bob then chooses either to **Meet** or **Pass**.
    - If he chooses to Pass, Alice wins the pot and the game ends.
    - If he chooses to Meet, he adds another $1 to the pot.
- There is then a showdown, in which Alice reveals her card.
    - If she has a King, then she wins the pot;
    - If she has a Queen, then Bob wins the pot.

In [80]:
import pygambit as gbt

Create the game with two players.

In [81]:
g = gbt.Game.new_tree(
    players=["Alice", "Bob"], 
    title="One card poker"
)

In addition to the two named players, Gambit also instantiates a chance player.

In [82]:
print(g.players["Alice"])
print(g.players["Bob"])
print(g.players.chance)

Player(game=Game(title='One card poker'), label='Alice')
Player(game=Game(title='One card poker'), label='Bob')
ChancePlayer(game=Game(title='One card poker'))


Moves belonging to the chance player can be added in the same way as to other players.

At any new move created for the chance player, the action probabilities default to uniform randomization over the actions at the move.

The first step in this game is that Alice is dealt a card which could be a King or Queen, each with probability 1/2.

To simulate this in Gambit, we create a chance player move at the root node of the game.

Note: throughout this tutorial, we'll also apply labels to the various nodes in the game tree to improve code readability.

In [83]:
g.append_move(
    g.root,
    player=g.players.chance,
    actions=["King", "Queen"]  # By default, chance actions have equal probabilities
)
g.root.children[0].label = "King"  # Add labels to the new child nodes to improve code readability
g.root.children[1].label = "Queen"

Now let's add Alice's first move after the card is dealt.

In this game, information structure is important.
Alice knows her card, so the two nodes at which she has the move are part of different information sets.

We'll therefore need to append Alice's move separately for each of the root node's children, i.e. the scenarios where she has a King or a Queen.

In [84]:
for node in g.root.children:
    g.append_move(
        node,
        player="Alice",
        actions=["Raise", "Fold"]
    )
    node.children[0].label = "Raise"
    node.children[1].label = "Fold"

The loop above causes each of the newly-appended moves to be in new **information sets**, reflecting the fact that Alice's decision depends on the knowledge of which card she holds.

In contrast, Bob does not know Alice’s card, and therefore cannot distinguish between the two nodes at which he has to make his decision:

  - Chance player chooses King, then Alice Raises: `g.root.children["King"].children["Raise"]`
  - Chance player chooses Queen, then Alice Raises: `g.root.children["Queen"].children["Raise"`

In other words, Bob's decision when Alice has a Queen should be part of the same information set as Bob's decision when Alice has a King.

To set this scenario up in Gambit, we'll need to use `Game.append_infoset` to add a move as part of an existing information set (represented in Gambit as an `Infoset`).

First, let's add Bob's move to the node where Alice has raised with a King.

In [85]:
g.append_move(
    g.root.children["King"].children["Raise"],
    player="Bob",
    actions=["Meet", "Pass"]
)
g.root.children["King"].children["Raise"].children[0].label = "Meet"
g.root.children["King"].children["Raise"].children[1].label = "Pass"

Now let's add the information set we created at the node where Alice raised with a King, to the node where Alice raised with a Queen.

In [86]:
g.append_infoset(
    g.root.children["Queen"].children["Raise"],
    infoset=g.root.children["King"].children["Raise"].infoset
)
g.root.children["Queen"].children["Raise"].children[0].label = "Meet"
g.root.children["Queen"].children["Raise"].children[1].label = "Pass"

In game theory terms, this creates "imperfect information".
Bob cannot distinguish between these two nodes in the game tree, so he must use the same strategy (same probabilities for Meet vs. Pass) in both situations.

This is crucial in games where players must make decisions without complete knowledge of their opponents' private information.

Let's now set up the four possible payoff outcomes for the game.

In [87]:
alice_winsbig = g.add_outcome([2, -2], label="Alice wins big")
alice_wins = g.add_outcome([1, -1], label="Alice wins")
bob_winsbig = g.add_outcome([-2, 2], label="Bob wins big")
bob_wins = g.add_outcome([-1, 1], label="Bob wins")

Finally, we should assign an outcome to each of the terminal nodes in the game tree.

In [88]:
# Alice folds, Bob wins small
g.set_outcome(g.root.children["King"].children["Fold"], bob_wins)
g.set_outcome(g.root.children["Queen"].children["Fold"], bob_wins)

# Bob sees Alice raise and calls, correctly believing she is bluffing, Bob wins big
g.set_outcome(g.root.children["Queen"].children["Raise"].children["Meet"], bob_winsbig)

# Bob sees Alice raise and calls, incorrectly believing she is bluffing, Alice wins big
g.set_outcome(g.root.children["King"].children["Raise"].children["Meet"], alice_winsbig)

# Bob does not call Alice's raise, Alice wins small
g.set_outcome(g.root.children["King"].children["Raise"].children["Pass"], alice_wins)
g.set_outcome(g.root.children["Queen"].children["Raise"].children["Pass"], alice_wins)

<a id="mye91"></a> Myerson, Roger B. (1991) *Game Theory: Analysis of Conflict*. Cambridge: Harvard University Press.

<a id="ruw08"></a> Reiley, David H., Michael B. Urbancic and Mark Walker. (2008) "Stripped-down poker: A classroom game with signaling and bluffing." *The Journal of Economic Education* 39(4): 323-341.

## Computing Nash equilibria

Since our one-card poker game is extensive form and has two players, we can use the `lcp_solve` algorithm in Gambit to compute the Nash equilibria.

In [89]:
result = gbt.nash.lcp_solve(g)
result

NashComputationResult(method='lcp', rational=True, use_strategic=False, equilibria=[[[[Rational(1, 1), Rational(0, 1)], [Rational(1, 3), Rational(2, 3)]], [[Rational(2, 3), Rational(1, 3)]]]], parameters={'stop_after': 0, 'max_depth': 0})

The result of the calculation is returned as a `NashComputationResult` object.

The set of equilibria found is reported in `NashComputationResult.equilibria`; in this case, this is a list of mixed behavior profiles.

For one-card poker, we expect to find a single equilibrium (one mixed behavior profile):

In [90]:
print("Number of equilibria found:", len(result.equilibria))
eqm = result.equilibria[0]

Number of equilibria found: 1


In [91]:
type(eqm)

pygambit.gambit.MixedBehaviorProfileRational

A mixed behavior profile specifies, for each information set, the probability distribution over actions at that information set.

Indexing a mixed behaviour profile by a player gives a `MixedBehavior`, which specifies probability distributions at each of the player's information sets:

In [116]:
type(eqm["Alice"])

pygambit.gambit.MixedBehavior

In [92]:
eqm["Alice"]

[[Rational(1, 1), Rational(0, 1)], [Rational(1, 3), Rational(2, 3)]]

In this case, at Alice's first information set, the one at which she has the King, she always raises.

At her second information set, where she has the Queen, she sometimes bluffs, raising with probability one-third.

The probability distribution at an information set is represented by a `MixedAction`.

`MixedBehavior.mixed_actions` iterates over these for the player:

In [128]:
for infoset, mixed_action in eqm["Alice"].mixed_actions():
    print(
        f"At information set {infoset.number}, "
        f"Alice plays Raise with probability: {mixed_action['Raise']}"
        f" and Fold with probability: {mixed_action['Fold']}"
    )

At information set 0, Alice plays Raise with probability: 1 and Fold with probability: 0
At information set 1, Alice plays Raise with probability: 1/3 and Fold with probability: 2/3


In larger games, labels may not always be the most convenient way to refer to specific actions.
We can also index profiles directly with `Action` objects.

So an alternative way to extract the probabilities of playing “Raise” would be by iterating Alice’s list of actions:

In [95]:
{action.infoset: eqm[action] for action in g.players["Alice"].actions if action.label == "Raise"}

{Infoset(player=Player(game=Game(title='One card poker'), label='Alice'), number=0): Rational(1, 1),
 Infoset(player=Player(game=Game(title='One card poker'), label='Alice'), number=1): Rational(1, 3)}

Now let's look at Bob’s strategy:

In [96]:
eqm["Bob"]

[[Rational(2, 3), Rational(1, 3)]]

Bob meets Alice’s raise two-thirds of the time.
The label “Raise” is used in more than one information set for Alice, so in the above we had to specify information sets when indexing.

When there is no ambiguity, we can specify action labels directly.
So for example, because Bob has only one action named “Meet” in the game, we can extract the probability that Bob plays “Meet” by:

In [97]:
eqm["Bob"]["Meet"]

Rational(2, 3)

Moreover, this is the only action with that label in the game, so we can index the profile directly using the action label without any ambiguity:

In [98]:
eqm["Meet"]

Rational(2, 3)

Because this is an equilibrium, the fact that Bob randomizes at his information set must mean he is indifferent between the two actions at his information set.
 
`MixedBehaviorProfile.action_value` returns the expected payoff of taking an action, conditional on reaching that action's information set:

In [99]:
{action.label: eqm.action_value(action) for action in g.players["Bob"].infosets[0].actions}

{'Meet': Rational(-1, 1), 'Pass': Rational(-1, 1)}

Bob's indifference between his actions arises because of his beliefs given Alice's strategy.
`MixedBehaviorProfile.belief` returns the probability of reaching a node, conditional on its information set being reached:

In [100]:
{node: eqm.belief(node) for node in g.players["Bob"].infosets[0].members}

{Node(game=Game(title='One card poker'), label='Raise'): Rational(3, 4),
 Node(game=Game(title='One card poker'), label='Raise'): Rational(1, 4)}

Bob believes that, conditional on Alice raising, there's a 3/4 chance that she has the King;
therefore, the expected payoff to meeting is in fact -1 as computed.
`MixedBehaviorProfile.infoset_prob` returns the probability that an information set is reached:

In [101]:
eqm.infoset_prob(g.players["Bob"].infosets[0])

Rational(2, 3)

The corresponding probability that a node is reached in the play of the game is given by `MixedBehaviorProfile.realiz_prob`, and the expected payoff to a player conditional on reaching a node is given by `MixedBehaviorProfile.node_value`.

In [102]:
{node: eqm.node_value("Bob", node) for node in g.players["Bob"].infosets[0].members}

{Node(game=Game(title='One card poker'), label='Raise'): Rational(-5, 3),
 Node(game=Game(title='One card poker'), label='Raise'): Rational(1, 1)}

The overall expected payoff to a player given the behavior profile is returned by `MixedBehaviorProfile.payoff`:

In [103]:
eqm.payoff("Alice")

Rational(1, 3)

In [104]:
eqm.payoff("Bob")

Rational(-1, 3)

The equilibrium computed expresses probabilities in rational numbers.

Because the numerical data of games in Gambit [are represented exactly](https://gambitproject.readthedocs.io/en/stable/pygambit.user.html#representation-of-numerical-data-of-a-game), methods which are specialized to two-player games, `lp_solve`, `lcp_solve`, and `enummixed_solve`, can report exact probabilities for equilibrium strategy profiles.

This is enabled by default for these methods.

When a game has an extensive representation, equilibrium finding methods default to computing on that representation.
It is also possible to compute using the strategic representation.
`pygambit` transparently computes the reduced strategic form representation of an extensive game.

In [105]:
[s.label for s in g.players["Alice"].strategies]

['11', '12', '21', '22']

In the strategic form of this game, Alice has four strategies.

The generated strategy labels list the action numbers taken at each information set.

We can therefore apply a method which operates on a strategic game to any game with an extensive representation.

In [106]:
result = gbt.nash.gnm_solve(g)
result

NashComputationResult(method='gnm', rational=False, use_strategic=True, equilibria=[[[0.33333333333866677, 0.6666666666613335, 0.0, 0.0], [0.6666666666559997, 0.3333333333440004]]], parameters={'perturbation': [[1.0, 0.0, 0.0, 0.0], [1.0, 0.0]], 'end_lambda': -10.0, 'steps': 100, 'local_newton_interval': 3, 'local_newton_maxits': 10})

`gnm_solve` can be applied to any game with any number of players, and uses a path-following process in floating-point arithmetic, so it returns profiles with probabilities expressed as floating-point numbers.

This method operates on the strategic representation of the game, so the returned results are of type `MixedStrategyProfile`, and specify, for each player, a probability distribution over that player's strategies.

Indexing a `MixedStrategyProfile` by a player gives the probability distribution over that player's strategies only.

In [107]:
eqm1 = result.equilibria[0]

In [108]:
eqm1["Alice"]

[0.33333333333866677, 0.6666666666613335, 0.0, 0.0]

In [109]:
eqm1["Bob"]

[0.6666666666559997, 0.3333333333440004]

The expected payoff to a strategy is provided by `MixedStrategyProfile.strategy_value`:

In [110]:
{strategy: eqm1.strategy_value(strategy) for strategy in g.players["Alice"].strategies}

{Strategy(player=Player(game=Game(title='One card poker'), label='Alice'), label='11'): 0.33333333334400045,
 Strategy(player=Player(game=Game(title='One card poker'), label='Alice'), label='12'): 0.33333333332799997,
 Strategy(player=Player(game=Game(title='One card poker'), label='Alice'), label='21'): -0.9999999999839995,
 Strategy(player=Player(game=Game(title='One card poker'), label='Alice'), label='22'): -1.0}

In [111]:
{strategy: eqm1.strategy_value(strategy) for strategy in g.players["Bob"].strategies}

{Strategy(player=Player(game=Game(title='One card poker'), label='Bob'), label='1'): -0.33333333333066656,
 Strategy(player=Player(game=Game(title='One card poker'), label='Bob'), label='2'): -0.3333333333386667}

The overall expected payoff to a player is returned by `MixedStrategyProfile.payoff`:

In [112]:
eqm1.payoff("Alice")

0.33333333333333354

In [113]:
eqm1.payoff("Bob")

-0.33333333333333354

When a game has an extensive representation, we can convert freely between `MixedStrategyProfile` and the corresponding `MixedBehaviorProfile` representation of the same strategies using `MixedStrategyProfile.as_behavior` and `MixedBehaviorProfile.as_strategy`.

In [114]:
eqm1.as_behavior()

[[[1.0, 0.0], [0.3333333333386667, 0.6666666666613333]], [[0.6666666666559997, 0.3333333333440004]]]

In [115]:
eqm1.as_behavior().as_strategy()

[[0.3333333333386667, 0.6666666666613333, 0.0, 0.0], [0.6666666666559997, 0.3333333333440004]]